<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=274b555e8fc25df3ef99c9ef16efaaffcebfaaf21bd31ac5eb0b51a10be2fefc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-20 14:25:08
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14


-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.36 C
-------------------
Today PnL: -2.84 L (-2.04%)
Current PnL: -32.77 L (-20.9%)
CY Booked + Current PnL: -18.48 L (-11.79%)
-------------------
Total profit:  1.35 L
Total loss:  -34.12 L
-------------------
Total Booked + Current PnL: 8.50 L (6.55%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.48%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 98.12 L (71.99%)
Deployed:  1.30 C
Current:  1.36 C
CAGR/XIRR %: 2.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.64,15.24,5.34,21.40,16334.0,40459.0,305885.0,6.96,58.0,X-LC,6.93,14.0,2.48,2.24,22.53,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,1.69,-23.31,30.51,0.09,23581.0,-23487.0,77290.0,79.30,37.0,M-SC,4.72,253.0,-1.00,0.57,1.69,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-2.87,2.74,16.04,19.23,24796.0,4126.0,154586.0,21.35,54.0,L-LC,10.38,182.0,0.17,1.13,24.60,XY25,ATH,MINING
49,MASFIN,398.61,-1.88,-6.71,30.82,22.05,28173.0,-6570.0,91410.0,-19.60,41.0,H-SC,4.97,168.0,-0.23,0.67,32.87,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.74,-18.96,23.42,0.02,33130.0,-33100.0,141459.0,115.96,41.0,M-SC,9.09,240.0,-1.00,1.04,23.05,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,1.69,-23.31,30.51,0.09,23581.0,-23487.0,77290.0,79.30,37.0,M-SC,4.72,253.0,-1.00,0.57,1.69,OX40N,NTT,DURABLES
28,GREENPANEL,537.00,-0.08,-43.88,155.06,43.14,159315.0,-80334.0,102744.0,129.97,27.0,M-SC,14.72,231.0,-0.50,0.75,0.00,XY24,NTT,MISC
33,HINDUNILVR,2922.00,-0.11,-6.20,21.18,13.67,51071.0,-15933.0,241130.0,-13.28,59.0,X-LC,2.37,9.0,-0.31,1.77,13.83,XY25,NTT,FMCG
34,HONAUT,58357.33,-0.28,-17.42,73.55,43.32,98925.0,-28372.0,134500.0,-29.05,47.0,X-SC,8.14,90.0,-0.29,0.99,3.27,X40N,ATH,ELECTRICAL
67,SONACOMS,806.63,-0.45,-22.90,81.00,39.54,63171.0,-23170.0,77989.0,-39.70,29.0,M-SC,3.18,227.0,-0.37,0.57,9.97,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HAVELLS,2069.16,-6.63,-13.49,53.15,32.48,150803.0,-44260.0,283731.0,-17.88,33.0,X-MC,5.14,69.0,-0.29,2.08,0.00,X40,ATH,ELECTRICAL
13,BSOFT,831.70,-5.26,-26.86,106.15,50.79,109207.0,-37773.0,102880.0,-6.07,41.0,H-SC,8.74,171.0,-0.35,0.75,19.79,XR,ATH,IT
40,INDUSINDBK,1800.00,-5.24,-33.55,100.00,32.89,51300.0,-25906.0,51300.0,-733.49,53.0,L-MC,10.17,259.0,-0.50,0.38,41.28,XR,NTT,BANKS
37,IEX,219.00,-4.88,-11.56,68.06,48.63,120351.0,-23123.0,176831.0,-41.22,38.0,H-SC,13.24,137.0,-0.19,1.30,0.00,XR,NTT,MISC
82,VOLTAS,1856.00,-4.63,4.34,39.16,45.20,78342.0,8313.0,200055.0,-5.78,35.0,X-MC,5.71,80.0,0.11,1.47,11.43,XY25,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.90,-7.00,123.57,107.91,169668.0,-10339.0,137305.0,-24.20,38.0,M-SC,9.93,220.0,-0.06,1.01,2.38,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.74,-18.96,23.42,0.02,33130.0,-33100.0,141459.0,115.96,41.0,M-SC,9.09,240.0,-1.00,1.04,23.05,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-3.24,-22.43,70.20,32.03,71792.0,-29567.0,102268.0,-16.04,48.0,H-SC,6.39,157.0,-0.41,0.75,14.55,OX40N,NTT,IT
77,TTKPRESTIG,770.00,1.69,-23.31,30.51,0.09,23581.0,-23487.0,77290.0,79.30,37.0,M-SC,4.72,253.0,-1.00,0.57,1.69,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.46,-24.23,49.77,13.47,101689.0,-65349.0,204318.0,-68.90,41.0,H-SC,4.23,159.0,-0.64,1.50,3.89,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-2.87,2.74,16.04,19.23,24796.0,4126.0,154586.0,21.35,54.0,L-LC,10.38,182.0,0.17,1.13,24.6,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-2.09,-1.23,74.42,72.27,124546.0,-2092.0,167356.0,-15.15,32.0,M-LC,5.11,101.0,-0.02,1.23,4.86,XR,NTT,IT
25,FINCABLES,1641.55,-2.90,-7.00,123.57,107.91,169668.0,-10339.0,137305.0,-24.20,38.0,M-SC,9.93,220.0,-0.06,1.01,2.38,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-2.10,-7.68,123.93,106.72,141107.0,-9476.0,113860.0,-54.25,44.0,H-SC,8.62,138.0,-0.07,0.84,13.90,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.84,-7.87,47.21,35.63,91157.0,-16486.0,193088.0,-20.73,34.0,H-MC,4.50,119.0,-0.18,1.42,7.49,AR,ATH,PHARMA
37,IEX,219.00,-4.88,-11.56,68.06,48.63,120351.0,-23123.0,176831.0,-41.22,38.0,H-SC,13.24,137.0,-0.19,1.30,0.00,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.96,-31.94,130.96,57.19,290678.0,-104168.0,221959.0,-67.84,16.0,X-MC,12.34,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
43,ITC,452.00,-1.46,-19.52,37.66,10.78,72834.0,-46920.0,193398.0,-52.19,17.0,X-LC,11.77,1.0,-0.64,1.42,0.0,X40,NTT,FMCG
58,RELAXO,1176.00,-3.03,-52.72,222.10,52.31,152447.0,-76521.0,68639.0,-49.67,18.0,X-SC,8.90,92.0,-0.50,0.50,0.0,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.77,-42.73,136.08,35.20,100280.0,-54978.0,73692.0,-3.39,21.0,X-SC,17.90,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-1.36,-10.11,49.28,34.19,94586.0,-21584.0,191936.0,-36.80,22.0,X-MC,5.12,60.0,-0.23,1.41,0.0,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,-1.41,-14.24,38.23,18.55,124013.0,-53851.0,324386.0,-26.21,45.0,X-LC,1.23,3.0,-0.43,2.38,10.36,X40,ATH,IT
20,DABUR,735.00,-1.38,-1.14,45.36,43.70,111241.0,-2828.0,245240.0,-9.24,45.0,X-MC,2.07,73.0,-0.03,1.80,13.74,XY24,BTT,FMCG
45,JIOFIN,387.00,-2.97,-13.97,44.70,24.48,102813.0,-37358.0,230007.0,-15.39,26.0,X-LC,2.13,37.0,-0.36,1.69,33.32,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,-0.50,-3.17,24.72,20.77,56048.0,-7414.0,226730.0,-11.53,33.0,X-MC,2.23,70.0,-0.13,1.66,3.53,X40N,NTT,BREWERIES
33,HINDUNILVR,2922.00,-0.11,-6.20,21.18,13.67,51071.0,-15933.0,241130.0,-13.28,59.0,X-LC,2.37,9.0,-0.31,1.77,13.83,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.96,-31.94,130.96,57.19,290678.0,-104168.0,221959.0,-67.84,16.0,X-MC,12.34,58.0,-0.36,1.63,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.77,-42.73,136.08,35.20,100280.0,-54978.0,73692.0,-3.39,21.0,X-SC,17.90,93.0,-0.55,0.54,0.0,X40,NTT,FOOTWEAR
9,BAJAJHLDNG,14451.52,-0.85,-5.60,36.89,29.22,66206.0,-10656.0,179469.0,-8.00,32.0,X-LC,12.23,29.0,-0.16,1.32,0.0,X40,ATH,FINANCE
8,BAJAJHFL,181.50,-1.42,-22.86,101.55,55.47,190573.0,-55622.0,187664.0,-31.69,24.0,X-MC,16.46,63.0,-0.29,1.38,0.0,X40N,ATH,FINANCE
31,HAVELLS,2069.16,-6.63,-13.49,53.15,32.48,150803.0,-44260.0,283731.0,-17.88,33.0,X-MC,5.14,69.0,-0.29,2.08,0.0,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.32,-32.47,111.51,42.84,48954.0,-21105.0,43901.0,-55.92,35.0,X-SC,23.36,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-3.03,-52.72,222.10,52.31,152447.0,-76521.0,68639.0,-49.67,18.0,X-SC,8.90,92.0,-0.50,0.50,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.77,-42.73,136.08,35.20,100280.0,-54978.0,73692.0,-3.39,21.0,X-SC,17.90,93.0,-0.55,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-3.26,-9.99,37.85,24.08,43250.0,-12682.0,114268.0,-17.20,31.0,X-SC,6.43,91.0,-0.29,0.84,7.67,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.28,-17.42,73.55,43.32,98925.0,-28372.0,134500.0,-29.05,47.0,X-SC,8.14,90.0,-0.29,0.99,3.27,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.46,-19.52,37.66,10.78,72834.0,-46920.0,193398.0,-52.19,17.0,X-LC,11.77,1.0,-0.64,1.42,0.00,X40,NTT,FMCG
75,TMPV,600.00,-1.69,-30.68,77.41,22.98,121214.0,-69311.0,156587.0,-27.96,31.0,X-LC,3.61,2.0,-0.57,1.15,0.00,XY24,NTT,AUTO
73,TCS,4311.59,-1.41,-14.24,38.23,18.55,124013.0,-53851.0,324386.0,-26.21,45.0,X-LC,1.23,3.0,-0.43,2.38,10.36,X40,ATH,IT
81,VBL,671.64,-1.08,-0.89,36.89,35.67,115479.0,-2807.0,313035.0,-12.64,51.0,X-LC,2.96,5.0,-0.02,2.30,12.84,X40N,ATH,FMCG
41,INFY,1972.00,-0.99,13.31,18.47,34.24,65483.0,41650.0,354538.0,-11.70,59.0,X-LC,7.40,6.0,0.64,2.60,22.70,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-5.24,-33.55,100.00,32.89,51300.0,-25906.0,51300.0,-733.49,53.0,L-MC,10.17,259.0,-0.50,0.38,41.28,XR,NTT,BANKS
17,COALINDIA,484.83,-2.87,2.74,16.04,19.23,24796.0,4126.0,154586.0,21.35,54.0,L-LC,10.38,182.0,0.17,1.13,24.60,XY25,ATH,MINING
36,ICICIPRULI,790.00,-0.86,8.43,20.56,30.73,43919.0,16607.0,213612.0,-16.91,46.0,X-MC,7.39,79.0,0.38,1.57,22.27,X40,ATH,INSURANCE
5,ASIANPAINT,3460.25,-2.31,-5.96,28.61,20.95,67736.0,-15013.0,236755.0,-6.96,38.0,X-LC,12.75,36.0,-0.22,1.74,27.55,X40,ATH,PAINTS
32,HCLTECH,1908.19,-1.56,9.67,12.91,23.83,34254.0,23406.0,265330.0,14.46,61.0,X-LC,6.36,13.0,0.68,1.95,27.30,X40,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-3.80,-30.59,62.67,12.90,95041.0,-66843.0,151653.0,123.56,54.0,H-SC,7.76,173.0,-0.70,1.11,26.10,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-2.87,2.74,16.04,19.23,24796.0,4126.0,154586.0,21.35,54.0,L-LC,10.38,182.0,0.17,1.13,24.60,XY25,ATH,MINING
62,SATIN,274.00,-1.21,-17.07,83.09,51.83,190371.0,-47170.0,229114.0,-25.10,58.0,H-SC,3.57,148.0,-0.25,1.68,11.61,XY24,NTT,FINANCE
33,HINDUNILVR,2922.00,-0.11,-6.20,21.18,13.67,51071.0,-15933.0,241130.0,-13.28,59.0,X-LC,2.37,9.0,-0.31,1.77,13.83,XY25,NTT,FMCG
40,INDUSINDBK,1800.00,-5.24,-33.55,100.00,32.89,51300.0,-25906.0,51300.0,-733.49,53.0,L-MC,10.17,259.0,-0.50,0.38,41.28,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.65
1,25,44.89
2,50,76.80


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.46
MC    30.66
LC    26.92
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.25
X40      24.19
X40N     12.92
XY25     10.95
XR        8.73
AR        8.40
OX40N     7.65
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.29
X-LC    23.51
H-SC    22.68
M-SC    11.11
X-SC     7.89
H-MC     4.54
M-MC     1.45
M-LC     1.23
L-LC     1.13
H-LC     1.05
L-SC     0.78
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.48,-10.58,46.86
IT,12.86,-22.02,82.97
FINANCE,11.09,-18.05,67.60
MISC,7.02,-35.81,91.36
PAINTS,5.81,-16.67,34.00
ELECTRICAL,5.77,-19.48,63.24
INSURANCE,4.76,-4.72,40.55
PHARMA,3.96,-8.98,43.29
BANKS,2.77,-41.89,127.87


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3395443.0,21
AR,1428413.0,10
XR,1300398.0,12
X40,1215616.0,15
X40N,982712.0,9
OX40N,749533.0,10
XY25,449388.0,7
SR,290876.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3701118.0,24
X-MC,1724876.0,16
M-SC,1547424.0,15
X-LC,1010801.0,13
X-SC,843773.0,8
H-MC,460731.0,3
L-SC,184469.0,2
M-LC,124546.0,1
H-LC,90670.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1295097.0      6
           AR         953105.0      5
M-SC       XY24       882740.0      6
H-SC       XR         840061.0      7
X-MC       X40        584292.0      7
           XY24       468461.0      3
           X40N       433396.0      4
X-LC       X40        430526.0      6
M-SC       OX40N      333691.0      5
X-SC       X40N       331058.0      3
H-SC       OX40N      321979.0      4
X-SC       XY24       311917.0      3
H-SC       SR         290876.0      2
H-MC       AR         247530.0      2
X-MC       XY25       238727.0      2
X-LC       XY24       224027.0      2
           X40N       218258.0      2
H-MC       XY24       213201.0      1
X-SC       X40        200798.0      2
M-SC       XR         193885.0      2
X-LC       XY25       137990.0      3
M-SC       AR         137108.0      2
M-LC       XR         124546.0      1
L-SC       OX40N       93863.0      1
H-LC       AR          90670.0      1
L-SC       XR          90606.0      1
L-MC       XR          51300.0      1
M-MC       XY25        47875.0      1
L-LC       XY25        24796.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
